In [28]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms

In [29]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [37]:
# Hyper-parametes
sequence_length = 28
input_size = 28
hidden_size = 128
num_layers = 2
num_classes = 10
batch_size = 100
num_epochs = 2
learning_rate = 0.003

In [38]:
# MNIST dataset
train_dataset = torchvision.datasets.MNIST(root='../../data/',
                                           train=True,
                                           transform=transforms.ToTensor(),
                                           download=True)

test_dataset = torchvision.datasets.MNIST(root='../../data/',
                                          train=False,
                                          transform=transforms.ToTensor())

In [39]:
# Data loader
train_loader = torch.utils.data.DataLoader(dataset = train_dataset,
                                           batch_size = batch_size,
                                           shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset = test_dataset,
                                          batch_size=batch_size,
                                          shuffle = True)

In [40]:
# BRNN (Bidirectional recurrent neural network (many-to-one))
class BiRNN(nn.Module):
  def __init__(self, input_size, hidden_size, num_layers, num_classes):
    super(BiRNN, self).__init__()
    self.hidden_size = hidden_size
    self.num_layers = num_layers
    self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True, bidirectional=True)
    self.fc = nn.Linear(hidden_size*2, num_classes)

  def forward(self, x):
    h0 = torch.zeros(self.num_layers*2, x.size(0), self.hidden_size).to(device)
    c0 = torch.zeros(self.num_layers*2, x.size(0), self.hidden_size).to(device)

    out, _ = self.lstm(x, (h0, c0)) # 마지막 스텝에 대한 정보를 가져옴 / out, _ 는 양방향 LSTM전달뜻인데 출력상태를 _에 저장하고있음

    out = self.fc(out[:, -1, :]) # 양방향 LSTM출력을 받아서 최종 출력을 만들기 위해 선형 레이어 통과시킴
    return out

In [41]:
# model 설정
model = BiRNN(input_size, hidden_size, num_layers, num_classes).to(device)

# Loss and Optimizer
criterion = nn.CrossEntropyLoss() # criterion(출력값, 실제레이블값)
optimizer = torch.optim.Adam(model.parameters(),lr=learning_rate)

In [42]:
# 모델 훈련
total_step = len(train_loader)
for epoch in range(num_epochs):
  for i, (images, labels) in enumerate(train_loader):
    images = images.reshape(-1, sequence_length, input_size).to(device)  # reshape(-1)을 함으로써 뒤의 매개변수들을 자동으로 조절해줌 / 이 경우에는 (28,28)로
    labels = labels.to(device)

    # 포워드
    outputs = model(images)
    loss = criterion(outputs, labels)

    # 백워드와 옵티마이저
    optimizer.zero_grad()
    loss.backward()  # 백워드를 태운다 = 모델의 손실에 대한 각 파라미터의 미분을 계산하는 과정을 거친다. / 여기서 각 파라미터는 가중치와 편향등 모델의 조절 가능한 변수들
    optimizer.step()

    if (i + 1) % 100 == 0:
      print('Epoch [{}/{}], Step[{}/{}], Loss:{:4f}'. format(epoch+1, num_epochs, i+1, total_step, loss.item()))

Epoch [1/2], Step[100/600], Loss:0.581801
Epoch [1/2], Step[200/600], Loss:0.251819
Epoch [1/2], Step[300/600], Loss:0.070859
Epoch [1/2], Step[400/600], Loss:0.186380
Epoch [1/2], Step[500/600], Loss:0.137295
Epoch [1/2], Step[600/600], Loss:0.081884
Epoch [2/2], Step[100/600], Loss:0.099439
Epoch [2/2], Step[200/600], Loss:0.122009
Epoch [2/2], Step[300/600], Loss:0.059047
Epoch [2/2], Step[400/600], Loss:0.105883
Epoch [2/2], Step[500/600], Loss:0.046276
Epoch [2/2], Step[600/600], Loss:0.007753


In [43]:
 # 모델 테스트
with torch.no_grad():# 그래디언트 계산을 중지 (학습이 완료되었기때문에 모델을 평가하거나 추론할때는 그래디언트 계산 x)
    correct = 0 # 맞은 수
    total = 0 # 전체 테스트 수
    for images, labels in test_loader:
      images = images.reshape(-1, sequence_length, input_size).to(device)
      labels = labels.to(device)
      outputs = model(images)
      _, predicted = torch.max(outputs.data, 1)    #torch.max 함수는 주어진 텐서에서 최댓값 최소값의 인덱스 반환
      # outputs.data 텐서의 두 번째 차원을 따라 최대값과 그 인덱스를 찾음
      # _ 는 최댓값 자체를 받지만, 이 값을 사용하지는 않음으로 무시한다는 뜻 / 즉 각 샘플이 속할 가능성이 가장 높은 클래스를 찾는 역할
      total += labels.size(0)
      correct += (predicted == labels).sum().item()   #.item()은  pt에서 단일 값을 Python숫자로 변환하는데 사용

    print('Test Accuracy of the model on the 10000 test images: {} %'.format(100 * correct / total))

# 모델 저장 체크포인트
torch.save(model.state_dict(), 'model.ckpt')


Test Accuracy of the model on the 10000 test images: 97.59 %
